In [2]:
import os
from pathlib import Path

import numpy as np
import tensorflow as tf
import keras
import pandas as pd

from frame_generator_kf_mv_v2 import FrameGenerator
from model_kf_mv_v2 import create_mv_model

2024-01-08 11:08:01.274251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 11:08:09.643721: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-08 11:08:09.643907: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-01-08 11:08:10.421630: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 11:08:20.861011: W tensorflow/stream_executor/platform/de

In [2]:
DATA_PATH = "data_right_left_up_down_1200"
NOTEBOOK_NAME = "4.3_dual_input_rlud_1200_kf_mv_v2"
RESULTS_PATH = DATA_PATH + "/" + NOTEBOOK_NAME

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

# PARAMS
n_frames = 74
batch_size = 8
HEIGHT=420
WIDTH = 10
MOTION_CHANNELS=1 # or 10 


assert(os.path.isdir(DATA_PATH ))

if not os.path.isdir(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)

index_df = pd.read_csv(f'{DATA_PATH}/indx_df.csv')

print(f"classes being compared {index_df['category'].unique()}")

classes being compared ['Pushing something from left to right'
 'Pushing something from right to left' 'Moving something up'
 'Moving something down']


In [3]:
subset_paths = {
    "test": Path(f'{DATA_PATH}/test'),
    "train": Path(f'{DATA_PATH}/train'),
    "val": Path(f'{DATA_PATH}/validation'),
}


output_signature = (
    (
        tf.TensorSpec(shape = (None, None, None, 3), dtype = tf.float32),
        tf.TensorSpec(shape = (n_frames, HEIGHT, WIDTH), dtype = tf.float32)
    ),
    tf.TensorSpec(shape = (), dtype = tf.int16)
)
train_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
train_ds = train_ds.batch(batch_size)

val_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
# Batch the data
val_ds = val_ds.batch(batch_size)

test_ds = tf.data.Dataset.from_generator(
    FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
    ),
    output_signature = output_signature
)
print(type(test_ds))

# Batch the data
test_ds = test_ds.batch(batch_size)

print(type(test_ds))


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>
<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>


In [4]:
data, labels = next(iter(train_ds))

2024-01-04 16:37:15.255646: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48168960 exceeds 10% of free system memory.


In [5]:
mv_input_shape = (n_frames, HEIGHT, WIDTH, MOTION_CHANNELS)
kf_input_shape = (None, None, None, None, 3)

model = create_mv_model(kf_input_shape, mv_input_shape)

In [6]:
model.build(data)

In [7]:
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer = keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=[
                    'accuracy',
                ]
            )

In [8]:
previous_runs = 1

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    RESULTS_PATH + '/model-runs-' + str(previous_runs) + '-cp-{epoch:02d}-{val_loss:.2f}.ckpt',
    save_best_only=True,  # Save only the best model based on a monitored metric (e.g., val_loss), will only replace saved value if it is better
    monitor='val_loss',
    mode='min',  # 'min' for loss, 'max' for accuracy
    save_weights_only=True
)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True
)

In [9]:
latest = tf.train.latest_checkpoint(RESULTS_PATH)

if latest is not None:
    print(f"loading model from weights: {latest}")
    model.load_weights(latest)

loading model from weights: data_right_left_up_down_1200/2_2d_plus_1_rlud_1200_kf_mv_v2/model-runs-1-cp-30-0.00.ckpt


2024-01-04 16:37:43.896098: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 246546432 exceeds 10% of free system memory.
2024-01-04 16:37:44.759666: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 246546432 exceeds 10% of free system memory.
2024-01-04 16:37:45.532001: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 246546432 exceeds 10% of free system memory.


In [ ]:
previously_run_epochs = 15 + 30

history = model.fit(
        x=train_ds,
        epochs = 50 - previously_run_epochs,
        validation_data=val_ds,
        callbacks=[checkpoint_callback, early_stopping],
    )

Epoch 1/5
480/480 [==============================] - 1151s 2s/step - loss: 5.7639e-06 - accuracy: 1.0000 - val_loss: 4.8195e-06 - val_accuracy: 1.0000
Epoch 2/5
480/480 [==============================] - 1142s 2s/step - loss: 4.6378e-06 - accuracy: 1.0000 - val_loss: 3.8715e-06 - val_accuracy: 1.0000
Epoch 3/5
480/480 [==============================] - 1138s 2s/step - loss: 3.7240e-06 - accuracy: 1.0000 - val_loss: 3.1034e-06 - val_accuracy: 1.0000
Epoch 4/5
 78/480 [===>..........................] - ETA: 9:03 - loss: 3.4912e-06 - accuracy: 1.0000

In [10]:
import numpy as np
from scipy.special import softmax
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

true_labels = []
predictions = []

# Iterate over the batched test dataset
for batch in test_ds:
    x, y = batch  # x is the batch of features, y is the batch of labels
    true_labels.extend(y.numpy())  # Store true labels
    preds = model.predict(x)  # Generate predictions for the batch
    preds = softmax(preds, axis=1)  # Apply softmax to convert logits to probabilities
    preds = np.argmax(preds, axis=1)  # Get the class with the highest probability
    predictions.extend(preds)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predictions = np.array(predictions)

2024-01-04 16:38:15.632959: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 48168960 exceeds 10% of free system memory.


1/1 [==============================] - 0s 267ms/step


In [11]:
fg = FrameGenerator(subset_paths['train'],
        n_frames=n_frames,
        height=HEIGHT,
        index_df=index_df,
        training=True
)
class_id_value = {
    fg.class_ids_for_name[x]: x for x in fg.class_ids_for_name.keys()
 }

# Convert lists to numpy arrays if they aren't already
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)

# Calculate precision, recall, and F1-score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average=None)

# Print accuracy and F1-scores for each class
print(f"Overall Accuracy: {accuracy}")
for i, (prec, rec, f1) in enumerate(zip(precision, recall, f1_score)):
    print(f"Class {class_id_value[i]}: Precision: {prec}, Recall: {rec}, F1 Score: {f1}")

Overall Accuracy: 1.0
Class Pushing something from left to right: Precision: 1.0, Recall: 1.0, F1 Score: 1.0
Class Pushing something from right to left: Precision: 1.0, Recall: 1.0, F1 Score: 1.0
Class Moving something up: Precision: 1.0, Recall: 1.0, F1 Score: 1.0
Class Moving something down: Precision: 1.0, Recall: 1.0, F1 Score: 1.0
